In [1]:
%cd ..

/Users/ahmedmaher/Desktop/product_classification_api


In [22]:
from datasets import load_dataset

# Data preprocessing

In [3]:
food_dataset=load_dataset('food101',split='train[:5000]')

In [4]:
food_splited=food_dataset.train_test_split(test_size=0.2)

In [5]:
food_splited

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1000
    })
})

In [6]:
labels=food_splited['train'].features['label'].names
label2id,id2label={},{}

In [7]:
for i,label in enumerate(labels):
    label2id[label]=i
    id2label[i]=label

# model Preprocessing

In [8]:
from transformers import AutoImageProcessor

In [9]:
processor=AutoImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k',use_fast=True)

In [10]:
from torchvision.transforms import RandomResizedCrop ,Compose,Normalize,Resize,ToTensor

In [11]:
size=(processor.size['height'],processor.size['width'])

In [12]:
transform=Compose([
    RandomResizedCrop(size),
    ToTensor(),
    Normalize(mean=processor.image_mean,std=processor.image_std)
])

In [13]:
def preprocess_image(samples):
    samples['pixel_values']=[transform(img.convert('RGB'))for img in samples['image']]
    del samples['image']
    return samples

In [14]:
food_splited_trans=food_splited.with_transform(preprocess_image)

In [15]:
food_splited_trans["train"][0]

{'label': 53,
 'pixel_values': tensor([[[ 0.8196,  0.7961,  0.7333,  ..., -0.6549, -0.6392, -0.5686],
          [ 0.7804,  0.7961,  0.7804,  ..., -0.7255, -0.6706, -0.6392],
          [ 0.7725,  0.7961,  0.8039,  ..., -0.7804, -0.7412, -0.7098],
          ...,
          [ 0.2000,  0.2314,  0.2471,  ...,  0.3255,  0.2706,  0.2157],
          [ 0.1765,  0.2235,  0.2314,  ...,  0.3255,  0.3176,  0.2863],
          [ 0.1686,  0.2314,  0.2157,  ...,  0.3020,  0.3098,  0.3098]],
 
         [[ 0.5059,  0.5059,  0.4588,  ..., -0.8431, -0.8510, -0.8118],
          [ 0.4902,  0.5137,  0.5216,  ..., -0.9137, -0.8745, -0.8667],
          [ 0.5059,  0.5137,  0.5451,  ..., -0.9529, -0.9294, -0.9059],
          ...,
          [ 0.2078,  0.2471,  0.2784,  ...,  0.3020,  0.2471,  0.2000],
          [ 0.1843,  0.2392,  0.2627,  ...,  0.3020,  0.2941,  0.2706],
          [ 0.1765,  0.2392,  0.2471,  ...,  0.2784,  0.2863,  0.2941]],
 
         [[ 0.0980,  0.0902,  0.0667,  ..., -0.9216, -0.9451, -0.9216]

In [16]:
from transformers import AutoModelForImageClassification, Trainer, TrainingArguments,DefaultDataCollator
data_collator=DefaultDataCollator()

In [17]:
import evaluate
import numpy as np
metric=evaluate.load("accuracy")
def compute_metrics(predictions_and_labels):
    predictions,labels=predictions_and_labels
    predictions=np.argmax(predictions,axis=1)
    return metric.compute(predictions=predictions,references=labels)

In [18]:
from transformers import logging
logging.set_verbosity_error()
model=AutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                      num_labels=len(labels),
                                                      id2label=id2label,
                                                      label2id=label2id
                                                     )


In [19]:
training_args=TrainingArguments(
    output_dir='my_model',
    remove_unused_columns=False,
    learning_rate=0.001,
     per_device_train_batch_size=16,
     per_device_eval_batch_size=16,
     num_train_epochs=32,
      weight_decay=0.01,
      save_strategy="epoch",
      logging_steps=10,
      eval_strategy="epoch",
     push_to_hub=False,
    metric_for_best_model="accuracy"

)

In [20]:
trainer=Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=food_splited_trans["train"],
    eval_dataset=food_splited_trans["test"]
)

In [21]:
trainer.train()

/Users/ahmedmaher/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 3.4625, 'grad_norm': 2.0701723098754883, 'learning_rate': 0.000998875, 'epoch': 0.04}
{'loss': 2.1775, 'grad_norm': 2.363891363143921, 'learning_rate': 0.000997625, 'epoch': 0.08}
{'loss': 1.9748, 'grad_norm': 1.380085825920105, 'learning_rate': 0.000996375, 'epoch': 0.12}
{'loss': 1.9265, 'grad_norm': 1.916972041130066, 'learning_rate': 0.0009951250000000001, 'epoch': 0.16}
{'loss': 1.8415, 'grad_norm': 2.3226826190948486, 'learning_rate': 0.000993875, 'epoch': 0.2}
{'loss': 1.8438, 'grad_norm': 2.0557971000671387, 'learning_rate': 0.000992625, 'epoch': 0.24}
{'loss': 1.8533, 'grad_norm': 1.5448344945907593, 'learning_rate': 0.000991375, 'epoch': 0.28}
{'loss': 1.833, 'grad_norm': 1.6742770671844482, 'learning_rate': 0.0009901250000000001, 'epoch': 0.32}
{'loss': 1.7798, 'grad_norm': 2.6234302520751953, 'learning_rate': 0.000988875, 'epoch': 0.36}
{'loss': 1.8778, 'grad_norm': 1.5299742221832275, 'learning_rate': 0.000987625, 'epoch': 0.4}
{'loss': 1.8116, 'grad_norm': 2.8321

/Users/ahmedmaher/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 1.6517, 'grad_norm': 1.8859647512435913, 'learning_rate': 0.000967625, 'epoch': 1.04}
{'loss': 1.8701, 'grad_norm': 2.7617526054382324, 'learning_rate': 0.000966375, 'epoch': 1.08}
{'loss': 1.8842, 'grad_norm': 2.812778949737549, 'learning_rate': 0.000965125, 'epoch': 1.12}
{'loss': 1.908, 'grad_norm': 2.0122551918029785, 'learning_rate': 0.0009638750000000001, 'epoch': 1.16}
{'loss': 1.9052, 'grad_norm': 1.2858878374099731, 'learning_rate': 0.0009626249999999999, 'epoch': 1.2}
{'loss': 1.8131, 'grad_norm': 2.1636619567871094, 'learning_rate': 0.000961375, 'epoch': 1.24}
{'loss': 1.8556, 'grad_norm': 2.0304503440856934, 'learning_rate': 0.000960125, 'epoch': 1.28}
{'loss': 1.8516, 'grad_norm': 1.9942653179168701, 'learning_rate': 0.0009588750000000001, 'epoch': 1.32}
{'loss': 1.7753, 'grad_norm': 0.9849271774291992, 'learning_rate': 0.0009576249999999999, 'epoch': 1.3599999999999999}
{'loss': 1.7541, 'grad_norm': 2.124354362487793, 'learning_rate': 0.000956375, 'epoch': 1.4}
{

/Users/ahmedmaher/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

/Users/ahmedmaher/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.7884, 'grad_norm': 2.2409002780914307, 'learning_rate': 6.1375e-05, 'epoch': 30.04}
{'loss': 0.779, 'grad_norm': 2.0917844772338867, 'learning_rate': 6.0125e-05, 'epoch': 30.08}
{'loss': 0.8056, 'grad_norm': 1.901414155960083, 'learning_rate': 5.8875e-05, 'epoch': 30.12}
{'loss': 0.7965, 'grad_norm': 1.4603939056396484, 'learning_rate': 5.7625000000000005e-05, 'epoch': 30.16}
{'loss': 0.7375, 'grad_norm': 1.6688029766082764, 'learning_rate': 5.6375e-05, 'epoch': 30.2}
{'loss': 0.7427, 'grad_norm': 2.2977092266082764, 'learning_rate': 5.5125e-05, 'epoch': 30.24}
{'loss': 0.6391, 'grad_norm': 1.6796159744262695, 'learning_rate': 5.3875e-05, 'epoch': 30.28}
{'loss': 0.79, 'grad_norm': 2.722339630126953, 'learning_rate': 5.2625e-05, 'epoch': 30.32}
{'loss': 0.715, 'grad_norm': 1.3753089904785156, 'learning_rate': 5.1374999999999995e-05, 'epoch': 30.36}
{'loss': 0.6922, 'grad_norm': 1.6689890623092651, 'learning_rate': 5.0125000000000005e-05, 'epoch': 30.4}
{'loss': 0.8301, 'grad

/Users/ahmedmaher/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.8392, 'grad_norm': 2.2213802337646484, 'learning_rate': 3.0125e-05, 'epoch': 31.04}
{'loss': 0.7224, 'grad_norm': 1.2882667779922485, 'learning_rate': 2.8875e-05, 'epoch': 31.08}
{'loss': 0.7573, 'grad_norm': 2.300048351287842, 'learning_rate': 2.7625e-05, 'epoch': 31.12}
{'loss': 0.7548, 'grad_norm': 1.8402055501937866, 'learning_rate': 2.6375e-05, 'epoch': 31.16}
{'loss': 0.7912, 'grad_norm': 1.851657509803772, 'learning_rate': 2.5125e-05, 'epoch': 31.2}
{'loss': 0.7781, 'grad_norm': 2.0750603675842285, 'learning_rate': 2.3875e-05, 'epoch': 31.24}
{'loss': 0.6917, 'grad_norm': 1.201710820198059, 'learning_rate': 2.2625e-05, 'epoch': 31.28}
{'loss': 0.6804, 'grad_norm': 1.460008978843689, 'learning_rate': 2.1375e-05, 'epoch': 31.32}
{'loss': 0.6675, 'grad_norm': 1.4523489475250244, 'learning_rate': 2.0125e-05, 'epoch': 31.36}
{'loss': 0.7854, 'grad_norm': 2.3688924312591553, 'learning_rate': 1.8875e-05, 'epoch': 31.4}
{'loss': 0.7991, 'grad_norm': 2.0799055099487305, 'learn

TrainOutput(global_step=8000, training_loss=1.1712001399993897, metrics={'train_runtime': 39000.4635, 'train_samples_per_second': 3.282, 'train_steps_per_second': 0.205, 'train_loss': 1.1712001399993897, 'epoch': 32.0})